Modifierad kod från första exemplet

In [156]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [205]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [    [54.4, 14.4, 'Kylrum'],    [45.4, 12.4, 'Kylrum'],    [89.4, 19.5, 'Klassrum'],
[57.4, 18.1, 'Lärarrum'],     [22.4, 8.6, 'Kylrum'],    [24.4, 11.24, 'Kylrum'],    [84.4, 24.4,
'Klassrum'],    [95.4, 22.4, 'Klassrum'],    [81.4, 20.1, 'Lärarrum'],    [70, 19.7, 'Lärarrum'],  [70, 19.8,'Klassrum']]

In [158]:
# Column labels.
# These are used only to print the tree.
header = ["luftfuktighet", "temperatur", "label"]

In [159]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [160]:
#######
# Demo:
unique_vals(training_data, 2)
# unique_vals(training_data, 1)
#######

{'Klassrum', 'Kylrum', 'Lärarrum'}

In [161]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [162]:
#######
# Demo:
class_counts(training_data)
#######

{'Kylrum': 4, 'Klassrum': 4, 'Lärarrum': 3}

In [163]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [164]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [165]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value, less, eq):
        self.column = column
        self.value  = value
        self.less   = less
        self.eq     = eq

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def moreless(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if self.less:
            if self.eq:
                return val <= self.value
            else:
                return val < self.value
        elif is_numeric(self.value):
            if self.eq:
                return val >= self.value
            else:
                return val > self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if self.less:
            if self.eq:
                condition = "<="
            else:
                condition = "<"
        elif is_numeric(self.value):
            if self.eq:
                condition = ">="
            else:
                condition = ">"
        return "Är %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [166]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(1, 19.5, 1, 0)

Är temperatur < 19.5?

In [167]:
# How about one for a categorical attribute
q = Question(0, 50, 1, 0)
q

Är luftfuktighet < 50?

In [168]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.moreless(example) # this will be true, since the first example is Green.
#######

False

In [169]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.moreless(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [170]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(1, 19.5, 1, 0))
# This will contain all the 'Red' rows.
true_rows

[[54.4, 14.4, 'Kylrum'],
 [45.4, 12.4, 'Kylrum'],
 [57.4, 18.1, 'Lärarrum'],
 [22.4, 8.6, 'Kylrum'],
 [24.4, 11.24, 'Kylrum']]

In [171]:
# This will contain everything else.
false_rows
#######

[[89.4, 19.5, 'Klassrum'],
 [84.4, 24.4, 'Klassrum'],
 [95.4, 22.4, 'Klassrum'],
 [81.4, 20.1, 'Lärarrum'],
 [70, 19.7, 'Lärarrum'],
 [70, 19.8, 'Klassrum']]

In [172]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [173]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [174]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [175]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [176]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [177]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6611570247933884

In [178]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(1, 19.5, 0, 1))
info_gain(true_rows, false_rows, current_uncertainty)

0.2732782369146006

In [179]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(1, 18.1, 1, 0))
info_gain(true_rows, false_rows, current_uncertainty)

0.3494687131050767

In [180]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(1, 19.7, 0, 0))

# Here, the true_rows contain only 'Grapes'.
true_rows

[[84.4, 24.4, 'Klassrum'],
 [95.4, 22.4, 'Klassrum'],
 [81.4, 20.1, 'Lärarrum'],
 [70, 19.8, 'Klassrum']]

In [181]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[[54.4, 14.4, 'Kylrum'],
 [45.4, 12.4, 'Kylrum'],
 [89.4, 19.5, 'Klassrum'],
 [57.4, 18.1, 'Lärarrum'],
 [22.4, 8.6, 'Kylrum'],
 [24.4, 11.24, 'Kylrum'],
 [70, 19.7, 'Lärarrum']]

In [182]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0, 45.4, 1, 1))

# We've isolated one apple in the true rows.
true_rows

[[45.4, 12.4, 'Kylrum'], [22.4, 8.6, 'Kylrum'], [24.4, 11.24, 'Kylrum']]

In [183]:
# But, the false-rows are badly mixed up.
false_rows
#######

[[54.4, 14.4, 'Kylrum'],
 [89.4, 19.5, 'Klassrum'],
 [57.4, 18.1, 'Lärarrum'],
 [84.4, 24.4, 'Klassrum'],
 [95.4, 22.4, 'Klassrum'],
 [81.4, 20.1, 'Lärarrum'],
 [70, 19.7, 'Lärarrum'],
 [70, 19.8, 'Klassrum']]

In [184]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            for less in range(0,2):
                for eq in range(0,2):
                    question = Question(col, val, less, eq)

                    # try splitting the dataset
                    true_rows, false_rows = partition(rows, question)

                    # Skip this split if it doesn't divide the
                    # dataset.
                    if len(true_rows) == 0 or len(false_rows) == 0:
                        continue

                    # Calculate the information gain from this split
                    gain = info_gain(true_rows, false_rows, current_uncertainty)

                    # You actually can use '>' instead of '>=' here
                    # but I wanted the tree to look a certain way for our
                    # toy dataset.
                    if gain > best_gain:
                        best_gain, best_question = gain, question

    return best_gain, best_question

In [201]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Är luftfuktighet > 54.4?

In [186]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [187]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [188]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [189]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [206]:
my_tree = build_tree(training_data)

In [207]:
print_tree(my_tree)

Är luftfuktighet > 54.4?
--> True:
  Är luftfuktighet > 81.4?
  --> True:
    Predict {'Klassrum': 3}
  --> False:
    Är temperatur >= 19.8?
    --> True:
      Är luftfuktighet >= 81.4?
      --> True:
        Predict {'Lärarrum': 1}
      --> False:
        Predict {'Klassrum': 1}
    --> False:
      Predict {'Lärarrum': 2}
--> False:
  Predict {'Kylrum': 4}


In [192]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.moreless(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [193]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Kylrum': 4}

In [194]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values())
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [195]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Kylrum': '100%'}

In [196]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Kylrum': '100%'}

In [209]:
# Evaluate
testing_data = [    [30, 15.6, 'Kylrum'],    [50, 16, 'Klassrum'],    [89, 21, 'Klassrum'],    [81.7, 19,
'Lärarrum'],    [87, 19.8, 'Lärarrum'],]

In [208]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Kylrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Kylrum': '100%'}
Actual: Klassrum. Predicted: {'Klassrum': '100%'}
Actual: Lärarrum. Predicted: {'Klassrum': '100%'}
Actual: Lärarrum. Predicted: {'Klassrum': '100%'}
